In [1]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [2]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [3]:
# determine user
user = getpass.getuser()
if user == 'peymanshahidi':
    main_folder_path = '/Users/peymanshahidi/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [4]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# occupation = 'athletesAndSportsCompetitors'
# occupation = 'dataEntryKeyer'
# occupation = 'audiovisualEquipmentInstallerAndRepairers'
# occupation = 'hearingAidSpecialists'
# occupation = 'dispatchersExceptPoliceFireAndAmbulance'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [5]:
# set alpha as AI quality metric
n = 100
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

### Initialize input-output paths

In [6]:
# Manual DAG
# input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'

# Naive DAG
# input_path = f'{occupation_folder}/{occupation}_N_GPT_DAG_df.csv'

# # Conditioned Naive DAG
input_path = f'{occupation_folder}/{occupation}_CN_GPT_DAG_df.csv'

# # First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'

# # Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'

In [7]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target,comment
0,Converse with customer to determine destinatio...,Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
1,"Compute cost of travel and accommodations, usi...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
2,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...",The worker booking transportation and hotel re...
3,"Plan, describe, arrange, and sell itinerary to...",Collect payment for transportation and accommo...,The worker collecting payment needs to know th...
5,Converse with customer to determine destinatio...,"Compute cost of travel and accommodations, usi...",The worker computing the cost of travel and ac...
6,Converse with customer to determine destinatio...,"Book transportation and hotel reservations, us...",The worker responsible for booking transportat...
7,Converse with customer to determine destinatio...,"Plan, describe, arrange, and sell itinerary to...",The worker planning and arranging travel packa...
9,Converse with customer to determine destinatio...,Print or request transportation carrier ticket...,The worker responsible for printing or request...
14,"Compute cost of travel and accommodations, usi...","Record and maintain information on clients, ve...","The worker maintaining information on clients,..."
15,"Book transportation and hotel reservations, us...","Plan, describe, arrange, and sell itinerary to...",The worker planning and selling itinerary tour...


In [8]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Collect payment for transportation and accommodations from customer.',
 1: 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 2: 'Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tours costs.',
 3: 'Book transportation and hotel reservations, using computer or telephone.',
 4: 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 5: 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 6: 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 7: 'Record and maintain information on clients, vendors, and travel packages.',
 8: '"Target"'}

In [9]:
print(f'in degree: {adjacency_matrix.sum(axis=0)}')
print(f'out degree: {adjacency_matrix.sum(axis=1)}')
print(f'\nsum in-out degree: {adjacency_matrix.sum(axis=0) + adjacency_matrix.sum(axis=1)}')

in degree: [3 0 1 2 2 1 1 2 4]
out degree: [2 5 2 1 3 1 1 1 0]

sum in-out degree: [5 5 3 3 5 2 2 3 4]


In [10]:
# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
#tasks_stats = tasks_stats[-5:]
tasks_stats = tasks_stats.reset_index(drop=True)
# tasks_stats

In [11]:
# add task_dict key and reset index
aux_dict = {value: key for key, value in tasks_dict.items()}
tasks_stats['dict_index'] = tasks_stats.apply(lambda row: aux_dict[row.task], axis=1)
tasks_stats = tasks_stats.sort_values(by='dict_index')
tasks_stats = tasks_stats.set_index('dict_index', drop=False)
tasks_stats.index.name = None
tasks_stats

,task,human_cost,management_cost,management_difficulty,machine_cost,completion_difficulty,dict_index
0,Collect payment for transportation and accommo...,15,5,0.25,300,0.30,0
1,Converse with customer to determine destinatio...,30,10,0.75,300,0.65,1
2,"Compute cost of travel and accommodations, usi...",45,10,0.65,300,0.30,2
3,"Book transportation and hotel reservations, us...",30,30,0.35,300,0.35,3
4,"Plan, describe, arrange, and sell itinerary to...",120,30,0.75,300,0.75,4
5,Provide customer with brochures and publicatio...,30,10,0.35,300,0.25,5
6,Print or request transportation carrier ticket...,10,5,0.25,300,0.25,6
7,"Record and maintain information on clients, ve...",30,5,0.35,300,0.35,7
8,"""Target""",0,0,0.00,0,0.00,8


In [12]:
tasks_stats

,task,human_cost,management_cost,management_difficulty,machine_cost,completion_difficulty,dict_index
0,Collect payment for transportation and accommo...,15,5,0.25,300,0.30,0
1,Converse with customer to determine destinatio...,30,10,0.75,300,0.65,1
2,"Compute cost of travel and accommodations, usi...",45,10,0.65,300,0.30,2
3,"Book transportation and hotel reservations, us...",30,30,0.35,300,0.35,3
4,"Plan, describe, arrange, and sell itinerary to...",120,30,0.75,300,0.75,4
5,Provide customer with brochures and publicatio...,30,10,0.35,300,0.25,5
6,Print or request transportation carrier ticket...,10,5,0.25,300,0.25,6
7,"Record and maintain information on clients, ve...",30,5,0.35,300,0.35,7
8,"""Target""",0,0,0.00,0,0.00,8


In [13]:
# create dictionaries for human cost, management cost, and difficulty
human_labor_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['human_cost']))
machine_labor_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['machine_cost']))
machine_management_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['management_cost']))

management_difficulty_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['management_difficulty']))
completion_difficulty_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['completion_difficulty']))

In [14]:
def find_neighbors(adjacency_matrix):
    # Get the number of nodes (n) from the shape of the adjacency matrix
    n = adjacency_matrix.shape[0]
    
    # Initialize an empty dictionary to store the neighbors for each node
    neighbors = {i: [] for i in range(n)}
    
    # Loop through each entry in the adjacency matrix
    for i in range(n):
        for j in range(n):
            # If there's an edge from i to j or from j to i, add j to the neighbors of i
            if adjacency_matrix[i, j] == 1 or adjacency_matrix[j, i] == 1:
                if j not in neighbors[i]:  # Avoid duplicate neighbors
                    neighbors[i].append(j)
                if i not in neighbors[j]:  # Ensure symmetry in the undirected version
                    neighbors[j].append(i)
    
    return neighbors

In [15]:
def create_inactive_node_neighbor_subset_combinations(inactive_neighbors_valid_subsets_dict):
    # Step 1: Extract unique lists from the dictionary values
    all_lists = [list(set(item)) for sublist in inactive_neighbors_valid_subsets_dict.values() for item in sublist]
    print(f'Number of lists extracted: {len(all_lists)}')

    # Step 2: Create all combinations and directly add unique elements
    output_set = set()
    
    # Instead of recomputing length and duplicates, work with unique sets directly
    all_combinations = []
    
    for r in range(1, len(all_lists) + 1):
        for combo in itertools.combinations(all_lists, r):
            # Convert each combination to a flattened tuple of sorted unique elements
            flattened_combo = tuple(sorted(set(itertools.chain(*combo))))
            output_set.add(flattened_combo)  # Add to set to ensure uniqueness
    
    # Convert the set back to sorted list of lists and return the result
    output_list = [list(combo) for combo in output_set]
    
    return sorted(output_list, key=len)

In [16]:
def is_connected(matrix):
    # Number of nodes in the matrix
    num_nodes = matrix.shape[0]
    
    # Visited array to keep track of visited nodes
    visited = np.zeros(num_nodes, dtype=bool)
    
    # Helper function to perform DFS
    def dfs(node):
        visited[node] = True
        # Visit all the neighbors of the current node
        for neighbor in range(num_nodes):
            if matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor)
            elif matrix[neighbor, node] == 1 and not visited[neighbor]:
                dfs(neighbor)
    
    # Start DFS from the first node (node 0)
    dfs(0)
    
    # If all nodes are visited, the matrix is connected
    return np.all(visited)


def validate_partition_using_connectedness(adjacency_matrix, tasks_list):
    # Return valid if Singleton
    if len(tasks_list) == 1:
        return True
    # Check if partition forms connected graph
    else:
        # Subset original adjacency matrix
        subset_matrix = adjacency_matrix[np.ix_(tasks_list, tasks_list)]

        # check if subset matrix is a connected graph
        subset_matrix_connected = is_connected(subset_matrix)

        # return true if connected and false otherwise
        return subset_matrix_connected

In [17]:
def get_partition_boundary(adjacency_matrix, partition):
    # create a matrix whose columns are nodes not in the partition and whose rows are nodes in the partition
    # (subset adjacency matrix to outgoing edges of partition nodes --i.e., rows-- and incoming edges of non-partition nodes --i.e., columns.)
    reduced_matrix = np.delete(adjacency_matrix, partition, axis=1) 
    reduced_matrix = reduced_matrix[partition, :]

    # find nodes in partition w/ an edge to non-partition nodes
    partition_boundary_tasks = [i for i in partition if np.any(reduced_matrix[partition.index(i), :])]

    return partition_boundary_tasks


def compute_plan_cost(adjacency_matrix,
                      execution_plan, 
                      human_labor_dict,
                      machine_labor_dict, machine_management_dict, 
                      management_difficulty_dict, completion_difficulty_dict,
                      AI_quality = 1e-8,
                      human_labor_wage = 10,
                      machine_management_wage = 10,
                      machine_automation_wage = 10):
    # initialize costs
    human_tasks_list = []
    managed_tasks_list = []
    automated_tasks_list = []

    total_cost = 0
    for partition in execution_plan:
        #print(f'Cost calculation partition: {partition}')
        if len(partition) == 1:
            human_labor_cost = sum(human_labor_dict[key] for key in partition)
            human_labor_cost_dollars = human_labor_cost * human_labor_wage
            
            machine_management_cost = sum(machine_management_dict[key] for key in partition)
            management_difficulty = sum(management_difficulty_dict[key] for key in partition)
            management_cost = machine_management_cost * (AI_quality ** (-1 * management_difficulty))
            management_cost_dollars = management_cost * machine_management_wage
            
            if human_labor_cost_dollars < management_cost_dollars:
                total_cost += human_labor_cost_dollars
                human_tasks_list.append(partition)
            if human_labor_cost_dollars >= management_cost_dollars:
                total_cost += management_cost_dollars
                managed_tasks_list.append(partition)
        else:
            # determine which tasks are automated and which tasks are managed
            managed_tasks = get_partition_boundary(adjacency_matrix, partition)
            automated_tasks = [task for task in partition if task not in managed_tasks]
            managed_tasks_list.append(managed_tasks)
            automated_tasks_list.append(automated_tasks)

            # calculate management cost of partition
            machine_management_cost = sum(machine_management_dict[key] for key in managed_tasks)
            management_difficulty = sum(management_difficulty_dict[key] for key in managed_tasks)
            management_cost = machine_management_cost * (AI_quality ** (-1 * management_difficulty))
            management_cost_dollars = management_cost * machine_management_wage
            total_cost += management_cost_dollars

            # calculate labor cost of partition
            machine_automation_cost = sum(machine_labor_dict[key] for key in automated_tasks)
            completion_difficulty = sum(completion_difficulty_dict[key] for key in automated_tasks)
            machine_cost = machine_automation_cost * (AI_quality ** (-1 * completion_difficulty))
            machine_cost_dollars = machine_cost * machine_automation_wage
            total_cost += machine_cost_dollars

    return total_cost

In [18]:
def anchor_min_cost(adjacency_matrix, 
                    human_labor_dict, machine_labor_dict, machine_management_dict, 
                    management_difficulty_dict, completion_difficulty_dict,
                    AI_quality):
    
    # find number of non-Target nodes
    n = adjacency_matrix[:-1,:-1].shape[0]
    
    # Candidate 1: All Singletons Plan
    all_singletons_plan = [[i] for i in range(n)]
    all_singletons_plan_cost = compute_plan_cost(adjacency_matrix, all_singletons_plan,
                                                 human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                 management_difficulty_dict, completion_difficulty_dict,
                                                 AI_quality)
    
    # print(f'All Singletons Plan: {all_singletons_plan}')
    # print(f'All Singletons Plan Cost: {all_singletons_plan_cost}\n')

    # Assign these to be the minimum cost and minimum cost plan
    min_cost_plan = all_singletons_plan
    min_cost = all_singletons_plan_cost
    

    
    # Candidate 2: All Bundled Plan
    all_bundled_plan = [[i for i in range(n)]]
    all_bundled_plan_cost = compute_plan_cost(adjacency_matrix, all_bundled_plan,
                                              human_labor_dict, machine_labor_dict, machine_management_dict, 
                                              management_difficulty_dict, completion_difficulty_dict,
                                              AI_quality)
    
    # print(f'All Bundled Plan: {all_bundled_plan}')
    # print(f'All Bundled Plan Cost: {all_bundled_plan_cost}\n')
    
    # Update minimum cost and minimum cost plan if necessary
    if all_bundled_plan_cost < min_cost:
        min_cost_plan = all_bundled_plan
        min_cost = all_bundled_plan_cost
    


    # Candidate 3: Immediate bundle of neighbor 0 and its immediate neighbors
    # initialize heuristic plan as all singletons plan
    heuristic_plan = all_singletons_plan

    # get list of node neighbors
    neighbors_dict = find_neighbors(adjacency_matrix[:-1,:-1]) # exclude Target node
    
    # iterate over nodes and update heuristic plan
    node = 0
    while node < n:
        # print(f'Node: {node}, Heuristic Plan: {heuristic_plan}')
        # get list of nodes that are in a size-two partition
        size_two_partitions = [sublist for sublist in heuristic_plan if len(sublist) == 2]
        size_two_partitions_flattened = [item for sublist in size_two_partitions for item in sublist]

        # if node is in a size-two partition, move to next node
        if node in size_two_partitions_flattened:
            node += 1
            continue

        # else, get neighbors of node
        neighbors_list = [neighbor for neighbor in neighbors_dict[node] if neighbor not in size_two_partitions_flattened and neighbor > node]

        if len(neighbors_list) == 0:
            node += 1
            continue
        
        # run through neighbors of node
        neighbor_min_cost_plan = []
        neighbor_min_cost = float('inf')
        for neighbor in neighbors_list:
            # print(f'Node: {node}, Neighbor: {neighbor}')
            # 1) create and calculate cost of singleton partition of node and neighbor
            singleton_partition = [[node], [neighbor]]
            singleton_partition_cost = compute_plan_cost(adjacency_matrix, singleton_partition,
                                                         human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                         management_difficulty_dict, completion_difficulty_dict,
                                                         AI_quality)
            
            # print(f'Singleton Partition: {singleton_partition}')
            # print(f'Singleton Partition Cost: {singleton_partition_cost}\n')

            # Update plan and cost if necessary
            if singleton_partition_cost < neighbor_min_cost:
                neighbor_min_cost_plan = singleton_partition
                neighbor_min_cost = singleton_partition_cost
            # print(f'Neighbor Min Cost Plan: {neighbor_min_cost_plan}\n')

            # 2) create and calculate cost of bundled partition of node and neighbor
            bundled_partition = [[node, neighbor]]
            bundled_partition_cost = compute_plan_cost(adjacency_matrix, bundled_partition,
                                                       human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                       management_difficulty_dict, completion_difficulty_dict,
                                                       AI_quality)
            
            # print(f'Bundled Partition: {bundled_partition}')
            # print(f'Bundled Partition Cost: {bundled_partition_cost}\n')
            
            # Update plan and cost if necessary
            if bundled_partition_cost < neighbor_min_cost:
                neighbor_min_cost_plan = bundled_partition
                neighbor_min_cost = bundled_partition_cost
            # print(f'Neighbor Min Cost Plan: {neighbor_min_cost_plan}\n')

        # substitute neighbor_min_cost_plan for optimal node-neighbor pair in heuristic plan
        # print(f'Neighbor Min Cost Plan: {neighbor_min_cost_plan}')
        # print(f'Neighbor Min Cost: {neighbor_min_cost}\n')
        neighbor_min_cost_plan_flattened = [item for sublist in neighbor_min_cost_plan for item in sublist]
        for my_node in neighbor_min_cost_plan_flattened:
            heuristic_plan.remove([my_node]) # remove singletons
        heuristic_plan += neighbor_min_cost_plan # add optimal partition

        # update node
        node += 1

    # calculate cost of optimal heuristic_plan
    heuristic_plan_cost = compute_plan_cost(adjacency_matrix, heuristic_plan,
                                            human_labor_dict, machine_labor_dict, machine_management_dict, 
                                            management_difficulty_dict, completion_difficulty_dict,
                                            AI_quality)
    
    # print(f'Optimal Heuristic Plan: {heuristic_plan}')
    # print(f'Optimal Heuristic Plan Cost: {heuristic_plan_cost}\n')

    # Update minimum cost and minimum cost plan if necessary
    if heuristic_plan_cost < min_cost:
        min_cost_plan = heuristic_plan
        min_cost = heuristic_plan_cost

    # return minimum cost plan and minimum cost
    return min_cost_plan, min_cost

In [19]:
def plan_to_active_dict(execution_plan, n):
    # Initialize a dictionary with n keys, all set to False
    init_dict = {i: False for i in range(n)}

    # Iterate through each sublist and each item in the list of lists
    for sublist in execution_plan:
        for item in sublist:
            # Keep only digits in the item
            key = ''.join(filter(str.isdigit, str(item)))
            if key.isdigit():  # Check if key is a valid digit
                key = int(key)
                if key in init_dict:  # Ensure key is within dictionary range
                    init_dict[key] = True
    return init_dict

In [20]:
def update_min_cost_vars(adjacency_matrix,
                         current_min_cost_plan, current_min_cost,
                         current_plan,
                         human_labor_dict, machine_labor_dict, machine_management_dict, 
                         management_difficulty_dict, completion_difficulty_dict,
                         AI_quality):
    
    # calculate cost of current plan
    current_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
                                            human_labor_dict, machine_labor_dict, machine_management_dict, 
                                            management_difficulty_dict, completion_difficulty_dict,
                                            AI_quality)
                    
    # if current_plan has lower cost than current_min_cost, update current_min_cost and current_min_cost_plan
    if current_plan_cost < current_min_cost:

        ######################################################## 
        # if costs are the same break tie in favor of more automation?
        ########################################################

        current_min_cost_plan = current_plan
        current_min_cost = current_plan_cost
        print('--------------------------------------')
        print('*minimum-cost execution plan updated*')
        print(f'new min cost plan: {current_min_cost_plan}')
        print(f'new min cost: {current_min_cost}')

    return current_min_cost_plan, current_min_cost

In [21]:
import copy
def get_min_cost_plan(adjacency_matrix, 
                      human_labor_dict, machine_labor_dict, machine_management_dict, 
                      management_difficulty_dict, completion_difficulty_dict,
                      AI_quality = 1e-8):

    def compute_min_cost_recursive(adjacency_matrix, 
                                   neighbors_dict, active_dict, memory_dict, 
                                   current_node, current_plan,
                                   current_min_cost_plan, current_min_cost,
                                   AI_quality):
        
        ############################################################################################################################
        def extended_plan_cost_check_and_append(adjacency_matrix, 
                                                human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                management_difficulty_dict, completion_difficulty_dict,
                                                current_plan, extended_plan,
                                                current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                neighbors_dict, active_dict, memory_dict,
                                                neighbor,
                                                current_min_cost_plan, current_min_cost,
                                                execution_plans_list,
                                                AI_quality):
            # check if need to pursue this plan
            extended_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
                                                    human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                    management_difficulty_dict, completion_difficulty_dict,
                                                    AI_quality)
            if extended_plan_cost < current_min_cost:
                # get acitve_dict for extended_plan
                active_dict = plan_to_active_dict(extended_plan, n)

                # calculate min cost and min cost plan for extended plan
                current_min_cost_plan , current_min_cost, execution_plan = compute_min_cost_recursive(adjacency_matrix, 
                                                                                                    neighbors_dict, active_dict, memory_dict, 
                                                                                                    [neighbor], extended_plan,
                                                                                                    current_min_cost_plan, current_min_cost,
                                                                                                    AI_quality)
                # sparse execution plan to update memory dict
                for plan in execution_plan:
                    exhausted_tasks = [item for sublist in plan for item in sublist]
                    if len(exhausted_tasks) != n:
                        continue

                    print('\n')
                    print(active_dict)
                    current_node_index = next(i for i, sublist in enumerate(plan) if current_node[0] in sublist)
                    print(f'current node = {current_node}')
                    print(f'current node index: {current_node_index}')
                    print(f'plan: {plan}')
                    extension = plan[current_node_index:]
                    print(f'extension of current plan {current_plan}: {extension}')
                    extension[0] = [item for item in extension[0] if item not in active_nodes_list_excluding_current_node]
                    print(f'modified extension of current plan {current_plan}: {extension}')

                    # update memory dict
                    memory_dict[(active_nodes_tuple, tuple(current_node))].append(extension)

                    # append plan to execution_plans_list
                    execution_plans_list.append(plan)
            else:
                print(f'not pursuing extended plan: {extended_plan}')
                print(f'extended plan cost: {extended_plan_cost} >= current min cost: {current_min_cost}\n')

            return current_min_cost_plan, current_min_cost, execution_plans_list
        ############################################################################################################################


        
        print('\n**************************************')
        print(f'current plan: {current_plan}')
        print(f'current node: {current_node}')
        print(f'active dict: {active_dict}\n')

        
        # "obvious" stopping rules: 
        # 1) if current_plan has higher cost than current_min_cost then break (important to appear hear because of neighbors of [0])
        current_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
                                              human_labor_dict, machine_labor_dict, machine_management_dict, 
                                              management_difficulty_dict, completion_difficulty_dict,
                                              AI_quality)
        if current_min_cost < current_plan_cost:
            print(f'not pursuing current plan: {current_plan}')
            print(f'current plan cost: {current_plan_cost} >= current min cost: {current_min_cost}\n')
            return current_min_cost_plan, current_min_cost, [current_plan]
        
        
        # get active nodes list
        active_nodes_list = [key for key, value in active_dict.items() if value == True]
        active_nodes_list_excluding_current_node = [item for item in active_nodes_list if item not in current_node]
        active_nodes_tuple = tuple(active_nodes_list_excluding_current_node)

        # get inactive neighbors of current_plan nodes
        current_plan_nodes = [item for sublist in current_plan for item in sublist]
        neighbors_list = list(dict.fromkeys([value for key in current_plan_nodes if key in neighbors_dict for value in neighbors_dict[key]]))
        print(f'neighbors of current plan:', neighbors_list)

        inactive_neighbors_list = [neighbor for neighbor in neighbors_list if active_dict[neighbor] == False]
        print(f'inactive neighbors of current plan:', inactive_neighbors_list)


        # if continuution of plan already calculated return it from memory
        try:
            if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) > 0:
                print('##########################################################################################')
                print(f'extentions of current node {current_node} already calculated:', memory_dict[(active_nodes_tuple, tuple(current_node))])
                print('##########################################################################################\n')

                if len(inactive_neighbors_list) == 0:
                    # if current_node is a last node, return current plan
                    if memory_dict[(active_nodes_tuple, tuple(current_node))][0] == []:
                        current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                                       current_min_cost_plan, current_min_cost,
                                                                                       current_plan,
                                                                                       human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                       management_difficulty_dict, completion_difficulty_dict,
                                                                                       AI_quality)
                        return current_min_cost_plan, current_min_cost, [current_plan]
                    else:
                        # generate all possible execution plans given extensions of current node
                        execution_plans_list = []
                        for extension in memory_dict[(active_nodes_tuple, tuple(current_node))]:
                            # to generate execution plan:
                            # 1) remove current node from last partition of current plan
                            # 2) extend modified last partition of current plan with first partition of current extension
                            # 3) add remaining partitions of current extension to the modified current plan
                            modified_current_plan_last_partition = copy.deepcopy(current_plan[-1])
                            modified_current_plan_last_partition.remove(current_node[0])
                            extension_first_partition = copy.deepcopy(extension[0])
                            modified_current_plan_last_partition += extension_first_partition

                            # create execution plan
                            execution_plan = current_plan[:-1] + [modified_current_plan_last_partition] + extension[1:]
                            execution_plans_list.append(execution_plan)
                            print(f'execution plan: {execution_plan}')

                            current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                                           current_min_cost_plan, current_min_cost,
                                                                                           execution_plan,
                                                                                           human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                           management_difficulty_dict, completion_difficulty_dict,
                                                                                           AI_quality)

                        return current_min_cost_plan, current_min_cost, execution_plans_list
                    
                else:
                    # print(f'execution_plans_list: {execution_plans_list}')
                    # print(f'current_plan: {current_plan}')

                    execution_plans_list = []
                    for neighbor in inactive_neighbors_list:
                        # print(f'inactive neighbor [{neighbor}] of current plan {current_plan} **********************************************')


                        #################################################################################################################################
                        #################################################################################################################################
                        print(f'\nextending current plan with inactive neighbor [{neighbor}]')

                        # extend current_plan to include inactive neighbor
                        # v1: add inactive neighbor as singletion partition to current_plan
                        extended_plan = copy.deepcopy(current_plan)
                        extended_plan += [[neighbor]]


            
                        current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                            human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                            management_difficulty_dict, completion_difficulty_dict,
                                                                                                                            current_plan, extended_plan,
                                                                                                                            current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                            neighbors_dict, active_dict, memory_dict,
                                                                                                                            neighbor,
                                                                                                                            current_min_cost_plan, current_min_cost,
                                                                                                                            execution_plans_list,
                                                                                                                            AI_quality)

                            
                        # v2: extend last partition in current_plan by adding inactive neighbor to it
                        extended_plan = copy.deepcopy(current_plan)
                        extended_plan[-1].append(neighbor)

                        # check validity of extended_plan
                        # (v2) extentions may not form a "valid" partition; if extension not valid skip 
                        if not validate_partition_using_connectedness(adjacency_matrix, extended_plan[-1]):
                            print(f'>>>>>>>>>>>>>>extended plan {extended_plan} not valid<<<<<<<<<<<<<')
                        else:
                            current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                                human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                                management_difficulty_dict, completion_difficulty_dict,
                                                                                                                                current_plan, extended_plan,
                                                                                                                                current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                                neighbors_dict, active_dict, memory_dict,
                                                                                                                                neighbor,
                                                                                                                                current_min_cost_plan, current_min_cost,
                                                                                                                                execution_plans_list,
                                                                                                                                AI_quality)

                    # hacky way of fixing no continuuation plan for current node:
                    if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) == 0:
                        memory_dict[(active_nodes_tuple, tuple(current_node))] = [[]]
                    
                    return current_min_cost_plan , current_min_cost, execution_plans_list
                    #################################################################################################################################
                    #################################################################################################################################







            
        # if partition not in memory, get valid subsets of partition
        except KeyError:
            # initialize memory dict key for current node
            memory_dict[(active_nodes_tuple, tuple(current_node))] = []

            if len(inactive_neighbors_list) == 0:
                # populate memory dict with current extension of inactive nodes
                extension = copy.deepcopy(current_plan[-1])
                extension.remove(current_node[0])
                inactive_extension = [item for item in extension if item not in active_nodes_list]
                memory_dict[(active_nodes_tuple, tuple(current_node))].append(inactive_extension)
                print(f'memory_dict updated for partition {current_node}: {memory_dict[(active_nodes_tuple, tuple(current_node))]}')

                # update min cost vars
                current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                               current_min_cost_plan, current_min_cost,
                                                                               current_plan,
                                                                               human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                               management_difficulty_dict, completion_difficulty_dict,
                                                                               AI_quality)
                
                return current_min_cost_plan, current_min_cost, [current_plan]
            
            
            execution_plans_list = []
            for neighbor in inactive_neighbors_list:
                print(f'\nextending current plan with inactive neighbor [{neighbor}]')

                # extend current_plan to include inactive neighbor
                # v1: add inactive neighbor as singletion partition to current_plan
                extended_plan = copy.deepcopy(current_plan)
                extended_plan += [[neighbor]]


    
                current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                    human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                    management_difficulty_dict, completion_difficulty_dict,
                                                                                                                    current_plan, extended_plan,
                                                                                                                    current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                    neighbors_dict, active_dict, memory_dict,
                                                                                                                    neighbor,
                                                                                                                    current_min_cost_plan, current_min_cost,
                                                                                                                    execution_plans_list,
                                                                                                                    AI_quality)

                    
                # v2: extend last partition in current_plan by adding inactive neighbor to it
                extended_plan = copy.deepcopy(current_plan)
                extended_plan[-1].append(neighbor)

                # check validity of extended_plan
                # (v2) extentions may not form a "valid" partition; if extension not valid skip 
                if not validate_partition_using_connectedness(adjacency_matrix, extended_plan[-1]):
                    print(f'>>>>>>>>>>>>>>extended plan {extended_plan} not valid<<<<<<<<<<<<<')
                else:
                    current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                        human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                        management_difficulty_dict, completion_difficulty_dict,
                                                                                                                        current_plan, extended_plan,
                                                                                                                        current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                        neighbors_dict, active_dict, memory_dict,
                                                                                                                        neighbor,
                                                                                                                        current_min_cost_plan, current_min_cost,
                                                                                                                        execution_plans_list,
                                                                                                                        AI_quality)

            # hacky way of fixing no continuuation plan for current node:
            if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) == 0:
                memory_dict[(active_nodes_tuple, tuple(current_node))] = [[]]

            return current_min_cost_plan , current_min_cost, execution_plans_list
    
    # subset adjacency matrix to exclude Target node
    non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()
    
    # get neighbors of nodes
    neighbors_dict = find_neighbors(non_target_adjacency_matrix)
    
    # get number of non-Target nodes
    n = non_target_adjacency_matrix.shape[0]

    # create active dictionary
    active_dict = plan_to_active_dict([[0]], n)
    
    # initialize dict for valid subsets of nodes (and also partitions) to act as memory
    memory_dict = {}

    # anchor min cost and min cost plan
    current_min_cost_plan, current_min_cost = anchor_min_cost(adjacency_matrix, 
                                                              human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                              management_difficulty_dict, completion_difficulty_dict,
                                                              AI_quality)
    print(f'Anchored min_cost_plan: {current_min_cost_plan}')
    print(f'Anchored min_cost: {current_min_cost}')

    min_cost_plan, min_cost, _ = compute_min_cost_recursive(adjacency_matrix, 
                                                         neighbors_dict, active_dict, memory_dict, 
                                                         [0], [[0]],
                                                         current_min_cost_plan, current_min_cost,
                                                         AI_quality)
    
    # indicator for whether memory_dict is used
    memory_dict_used = len(memory_dict) > 0
    
    return min_cost_plan, min_cost, memory_dict, memory_dict_used

In [22]:
example_adjacency_matrix = np.array([[0, 1, 1, 0, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 0, 0],
                                     [0, 0, 0, 0, 0]])

# print(f'in degree: {example_adjacency_matrix.sum(axis=0)}')
# print(f'out degree: {example_adjacency_matrix.sum(axis=1)}\n')

min_cost_plan, min_cost, memory_dict, memory_dict_used = get_min_cost_plan(example_adjacency_matrix, 
                                                         human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                         management_difficulty_dict, completion_difficulty_dict,
                                                         AI_quality = 1e3)#epsilon)
min_cost_plan, min_cost, memory_dict, memory_dict_used

Anchored min_cost_plan: [[0, 1, 2, 3]]
Anchored min_cost: 0.1901871830953335

**************************************
current plan: [[0]]
current node: [0]
active dict: {0: True, 1: False, 2: False, 3: False}

not pursuing current plan: [[0]]
current plan cost: 8.891397050194614 >= current min cost: 0.1901871830953335



([[0, 1, 2, 3]], 0.1901871830953335, {}, False)

In [23]:
# example_adjacency_matrix = np.array([[0, 1, 1, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 0],
#                                      [0, 0, 0, 0, 0]])

# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

In [24]:
# example_adjacency_matrix = np.array([[0, 0, 1, 0, 0],
#                                      [0, 0, 1, 0, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 1],
#                                      [0, 0, 0, 0, 0]])


# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

## Remove "Prints" for the test run on occupation with n=11 tasks

In [25]:
# def update_min_cost_vars(adjacency_matrix,
#                          current_min_cost_plan, current_min_cost,
#                          current_plan,
#                          human_labor_dict, machine_labor_dict, machine_management_dict, 
#                          management_difficulty_dict, completion_difficulty_dict,
#                          AI_quality):
    
#     # calculate cost of current plan
#     current_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
#                                             human_labor_dict, machine_labor_dict, machine_management_dict, 
#                                             management_difficulty_dict, completion_difficulty_dict,
#                                             AI_quality)
                    
#     # if current_plan has lower cost than current_min_cost, update current_min_cost and current_min_cost_plan
#     if current_plan_cost < current_min_cost:

#         ######################################################## 
#         # if costs are the same break tie in favor of more automation?
#         ########################################################

#         current_min_cost_plan = current_plan
#         current_min_cost = current_plan_cost

#     return current_min_cost_plan, current_min_cost



import copy
def get_min_cost_plan(adjacency_matrix, 
                      human_labor_dict, machine_labor_dict, machine_management_dict, 
                      management_difficulty_dict, completion_difficulty_dict,
                      AI_quality = 1e-8):

    def compute_min_cost_recursive(adjacency_matrix, 
                                   neighbors_dict, active_dict, memory_dict, 
                                   current_node, current_plan,
                                   current_min_cost_plan, current_min_cost,
                                   AI_quality):
        
        ############################################################################################################################
        def extended_plan_cost_check_and_append(adjacency_matrix, 
                                                human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                management_difficulty_dict, completion_difficulty_dict,
                                                current_plan, extended_plan,
                                                current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                neighbors_dict, active_dict, memory_dict,
                                                neighbor,
                                                current_min_cost_plan, current_min_cost,
                                                execution_plans_list,
                                                AI_quality):
            # check if need to pursue this plan
            extended_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
                                                    human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                    management_difficulty_dict, completion_difficulty_dict,
                                                    AI_quality)
            if extended_plan_cost < current_min_cost:
                # get acitve_dict for extended_plan
                active_dict = plan_to_active_dict(extended_plan, n)

                # calculate min cost and min cost plan for extended plan
                current_min_cost_plan , current_min_cost, execution_plan = compute_min_cost_recursive(adjacency_matrix, 
                                                                                                    neighbors_dict, active_dict, memory_dict, 
                                                                                                    [neighbor], extended_plan,
                                                                                                    current_min_cost_plan, current_min_cost,
                                                                                                    AI_quality)
                # sparse execution plan to update memory dict
                for plan in execution_plan:
                    exhausted_tasks = [item for sublist in plan for item in sublist]
                    if len(exhausted_tasks) != n:
                        continue

                    
                    current_node_index = next(i for i, sublist in enumerate(plan) if current_node[0] in sublist)
                    extension = plan[current_node_index:]
                    extension[0] = [item for item in extension[0] if item not in active_nodes_list_excluding_current_node]
                    
                    # update memory dict
                    memory_dict[(active_nodes_tuple, tuple(current_node))].append(extension)

                    # append plan to execution_plans_list
                    execution_plans_list.append(plan)

            return current_min_cost_plan, current_min_cost, execution_plans_list
        ############################################################################################################################


        # "obvious" stopping rules: 
        # 1) if current_plan has higher cost than current_min_cost then break (important to appear hear because of neighbors of [0])
        current_plan_cost = compute_plan_cost(adjacency_matrix, current_plan, 
                                              human_labor_dict, machine_labor_dict, machine_management_dict, 
                                              management_difficulty_dict, completion_difficulty_dict,
                                              AI_quality)
        if current_min_cost < current_plan_cost:
            return current_min_cost_plan, current_min_cost, [current_plan]
        
        
        # get active nodes list
        active_nodes_list = [key for key, value in active_dict.items() if value == True]
        active_nodes_list_excluding_current_node = [item for item in active_nodes_list if item not in current_node]
        active_nodes_tuple = tuple(active_nodes_list_excluding_current_node)

        # get inactive neighbors of current_plan nodes
        current_plan_nodes = [item for sublist in current_plan for item in sublist]
        neighbors_list = list(dict.fromkeys([value for key in current_plan_nodes if key in neighbors_dict for value in neighbors_dict[key]]))

        inactive_neighbors_list = [neighbor for neighbor in neighbors_list if active_dict[neighbor] == False]


        # if continuution of plan already calculated return it from memory
        try:
            if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) > 0:

                # print(f'extentions of current node {current_node} already calculated:', memory_dict[(active_nodes_tuple, tuple(current_node))])
                
                if len(inactive_neighbors_list) == 0:
                    # if current_node is a last node, return current plan
                    if memory_dict[(active_nodes_tuple, tuple(current_node))][0] == []:
                        current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                                       current_min_cost_plan, current_min_cost,
                                                                                       current_plan,
                                                                                       human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                       management_difficulty_dict, completion_difficulty_dict,
                                                                                       AI_quality)
                        return current_min_cost_plan, current_min_cost, [current_plan]
                    else:
                        # generate all possible execution plans given extensions of current node
                        execution_plans_list = []
                        for extension in memory_dict[(active_nodes_tuple, tuple(current_node))]:
                            # to generate execution plan:
                            # 1) remove current node from last partition of current plan
                            # 2) extend modified last partition of current plan with first partition of current extension
                            # 3) add remaining partitions of current extension to the modified current plan
                            modified_current_plan_last_partition = copy.deepcopy(current_plan[-1])
                            modified_current_plan_last_partition.remove(current_node[0])
                            extension_first_partition = copy.deepcopy(extension[0])
                            modified_current_plan_last_partition += extension_first_partition

                            # create execution plan
                            execution_plan = current_plan[:-1] + [modified_current_plan_last_partition] + extension[1:]
                            execution_plans_list.append(execution_plan)
                            
                            current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                                           current_min_cost_plan, current_min_cost,
                                                                                           execution_plan,
                                                                                           human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                           management_difficulty_dict, completion_difficulty_dict,
                                                                                           AI_quality)

                        return current_min_cost_plan, current_min_cost, execution_plans_list
                    
                else:
                    # print(f'execution_plans_list: {execution_plans_list}')
                    # print(f'current_plan: {current_plan}')

                    execution_plans_list = []
                    for neighbor in inactive_neighbors_list:
                        # print(f'inactive neighbor [{neighbor}] of current plan {current_plan} **********************************************')


                        #################################################################################################################################
                        #################################################################################################################################
                        
                        # extend current_plan to include inactive neighbor
                        # v1: add inactive neighbor as singletion partition to current_plan
                        extended_plan = copy.deepcopy(current_plan)
                        extended_plan += [[neighbor]]
                        # extended_plan = [sorted(sublist) for sublist in extended_plan]


            
                        current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                            human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                            management_difficulty_dict, completion_difficulty_dict,
                                                                                                                            current_plan, extended_plan,
                                                                                                                            current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                            neighbors_dict, active_dict, memory_dict,
                                                                                                                            neighbor,
                                                                                                                            current_min_cost_plan, current_min_cost,
                                                                                                                            execution_plans_list,
                                                                                                                            AI_quality)

                            
                        # v2: extend last partition in current_plan by adding inactive neighbor to it
                        extended_plan = copy.deepcopy(current_plan)
                        extended_plan[-1].append(neighbor)
                        # extended_plan = [sorted(sublist) for sublist in extended_plan]

                        # check validity of extended_plan
                        # (v2) extentions may not form a "valid" partition; if extension not valid skip 
                        if not validate_partition_using_connectedness(adjacency_matrix, extended_plan[-1]):
                            aaa = 1
                        else:
                            current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                                human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                                management_difficulty_dict, completion_difficulty_dict,
                                                                                                                                current_plan, extended_plan,
                                                                                                                                current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                                neighbors_dict, active_dict, memory_dict,
                                                                                                                                neighbor,
                                                                                                                                current_min_cost_plan, current_min_cost,
                                                                                                                                execution_plans_list,
                                                                                                                                AI_quality)

                    # hacky way of fixing no continuuation plan for current node:
                    if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) == 0:
                        memory_dict[(active_nodes_tuple, tuple(current_node))] = [[]]
                    
                    return current_min_cost_plan , current_min_cost, execution_plans_list
                    #################################################################################################################################
                    #################################################################################################################################







            
        # if partition not in memory, get valid subsets of partition
        except KeyError:
            # initialize memory dict key for current node
            memory_dict[(active_nodes_tuple, tuple(current_node))] = []

            if len(inactive_neighbors_list) == 0:
                # populate memory dict with current extension of inactive nodes
                extension = copy.deepcopy(current_plan[-1])
                extension.remove(current_node[0])
                inactive_extension = [item for item in extension if item not in active_nodes_list]
                memory_dict[(active_nodes_tuple, tuple(current_node))].append(inactive_extension)
                
                # update min cost vars
                current_min_cost_plan, current_min_cost = update_min_cost_vars(adjacency_matrix,
                                                                                current_min_cost_plan, current_min_cost,
                                                                                current_plan,
                                                                                human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                management_difficulty_dict, completion_difficulty_dict,
                                                                                AI_quality)
                
                return current_min_cost_plan, current_min_cost, [current_plan]
            
            
            execution_plans_list = []
            for neighbor in inactive_neighbors_list:
                
                # extend current_plan to include inactive neighbor
                # v1: add inactive neighbor as singletion partition to current_plan
                extended_plan = copy.deepcopy(current_plan)
                extended_plan += [[neighbor]]
                # extended_plan = [sorted(sublist) for sublist in extended_plan]



                current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                    human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                    management_difficulty_dict, completion_difficulty_dict,
                                                                                                                    current_plan, extended_plan,
                                                                                                                    current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                    neighbors_dict, active_dict, memory_dict,
                                                                                                                    neighbor,
                                                                                                                    current_min_cost_plan, current_min_cost,
                                                                                                                    execution_plans_list,
                                                                                                                    AI_quality)

                    
                # v2: extend last partition in current_plan by adding inactive neighbor to it
                extended_plan = copy.deepcopy(current_plan)
                extended_plan[-1].append(neighbor)
                # extended_plan = [sorted(sublist) for sublist in extended_plan]

                # check validity of extended_plan
                # (v2) extentions may not form a "valid" partition; if extension not valid skip 
                if not validate_partition_using_connectedness(adjacency_matrix, extended_plan[-1]):
                    aaa = 1
                else:
                    current_min_cost_plan, current_min_cost, execution_plans_list = extended_plan_cost_check_and_append(adjacency_matrix, 
                                                                                                                        human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                                                                        management_difficulty_dict, completion_difficulty_dict,
                                                                                                                        current_plan, extended_plan,
                                                                                                                        current_node, active_nodes_list_excluding_current_node, active_nodes_tuple,
                                                                                                                        neighbors_dict, active_dict, memory_dict,
                                                                                                                        neighbor,
                                                                                                                        current_min_cost_plan, current_min_cost,
                                                                                                                        execution_plans_list,
                                                                                                                        AI_quality)

            # hacky way of fixing no continuuation plan for current node:
            if len(memory_dict[(active_nodes_tuple, tuple(current_node))]) == 0:
                memory_dict[(active_nodes_tuple, tuple(current_node))] = [[]]

            return current_min_cost_plan , current_min_cost, execution_plans_list
    
    # subset adjacency matrix to exclude Target node
    non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()
    
    # get neighbors of nodes
    neighbors_dict = find_neighbors(non_target_adjacency_matrix)
    
    # get number of non-Target nodes
    n = non_target_adjacency_matrix.shape[0]

    # create active dictionary
    active_dict = plan_to_active_dict([[0]], n)
    
    # initialize dict for valid subsets of nodes (and also partitions) to act as memory
    memory_dict = {}

    # anchor min cost and min cost plan
    current_min_cost_plan, current_min_cost = anchor_min_cost(adjacency_matrix, 
                                                              human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                              management_difficulty_dict, completion_difficulty_dict,
                                                              AI_quality)
    print(f'Anchored min_cost_plan: {current_min_cost_plan}')
    print(f'Anchored min_cost: {current_min_cost}')

    #current_min_cost_plan, current_min_cost = [], float('inf')
    min_cost_plan, min_cost, _ = compute_min_cost_recursive(adjacency_matrix, 
                                                         neighbors_dict, active_dict, memory_dict, 
                                                         [0], [[0]],
                                                         current_min_cost_plan, current_min_cost,
                                                         AI_quality)
    
    # indicator for whether memory_dict is used
    memory_dict_used = len(memory_dict) > 0
    
    return min_cost_plan, min_cost, memory_dict, memory_dict_used

In [26]:
adjacency_matrix.shape # inclduing "Target" node

(9, 9)

In [27]:
min_cost_plan, min_cost, memory_dict, memory_dict_used = get_min_cost_plan(adjacency_matrix, 
                                                                           human_labor_dict, machine_labor_dict, machine_management_dict, 
                                                                           management_difficulty_dict, completion_difficulty_dict,
                                                                           AI_quality = epsilon)
min_cost_plan, min_cost, memory_dict, memory_dict_used

Anchored min_cost_plan: [[0], [1], [6], [2], [7], [3], [4], [5]]
Anchored min_cost: 3100


([[0], [1], [6], [2], [7], [3], [4], [5]],
 3100,
 {((), (0,)): [[[0], [1], [2], [3], [4], [6], [7], [5]],
   [[0], [1], [2], [3], [4], [6], [5], [7]],
   [[0], [1], [2], [3], [4], [7], [6], [5]],
   [[0], [1], [2], [3], [4], [7], [5], [6]],
   [[0], [1], [2], [3], [4], [5], [6], [7]],
   [[0], [1], [2], [3], [4], [5], [7], [6]],
   [[0], [1], [2], [3], [6], [4], [7], [5]],
   [[0], [1], [2], [3], [6], [4], [5], [7]],
   [[0], [1], [2], [3], [6], [7], [4], [5]],
   [[0], [1], [2], [3], [6], [7], [4, 5]],
   [[0], [1], [2], [3], [7], [4], [6], [5]],
   [[0], [1], [2], [3], [7], [4], [5], [6]],
   [[0], [1], [2], [3], [7], [6], [4], [5]],
   [[0], [1], [2], [3], [7], [6], [4, 5]],
   [[0], [1], [2], [4], [3], [6], [7], [5]],
   [[0], [1], [2], [4], [3], [6], [5], [7]],
   [[0], [1], [2], [4], [3], [7], [6], [5]],
   [[0], [1], [2], [4], [3], [7], [5], [6]],
   [[0], [1], [2], [4], [3], [5], [6], [7]],
   [[0], [1], [2], [4], [3], [5], [7], [6]],
   [[0], [1], [2], [4], [6], [3], [7], [5]

In [28]:
if memory_dict_used:
    print(len(memory_dict[(), (0,)]))
else:
    print('memory dict not used')

1080
